In [ ]:
!pip install streamlit -q
!pip install pyngrok

In [ ]:
!pip install fastapi uvicorn pydantic requests pyngrok nest-asyncio

In [ ]:
!pip install diffusers transformers accelerate scipy safetensors

In [ ]:
#!pip install gunicorn

In [ ]:
!pip install torch pillow

In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token('2Shc5Xi1V1t2TqXX5RkZsntEVOS_2EbpZ4quiWLkj3AsjJGeA')

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn
import json
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio

import requests
import numpy as np
import torch
from pydantic import BaseModel
from typing import List
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from torch import autocast


In [ ]:
app = FastAPI()

In [ ]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [ ]:
model_id = "stabilityai/stable-diffusion-2-1"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float32)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")

In [ ]:
class Item(BaseModel):
    prompt_id : str
    prompt_detail : str

In [ ]:
import cv2
import numpy as np

def cartoonify(input_path, output_path):
    # 이미지 로드
    image = cv2.imread(input_path)
    if image is None:
        raise ValueError("이미지를 로드할 수 없습니다.")

    # 이미지 사이즈 조정
    scaled_width = 600
    ratio = image.shape[1] / scaled_width
    scaled_height = int(image.shape[0] / ratio)
    image = cv2.resize(image, (scaled_width, scaled_height))

    # 블러 적용하여 노이즈 제거 : 경계선 부드러워짐
    blurred = cv2.medianBlur(image, 5)

    # 그레이스케일 변환
    gray = cv2.cvtColor(blurred, cv2.COLOR_BGR2GRAY)

    # 경계선 감지 : 두꺼운 경계선
    edges = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 9)

    # 컬러 이미지로 변환 : 색상이 더 부드러워짐
    color = cv2.bilateralFilter(image, 4, 250, 250)

    # 경계선과 컬러 이미지를 결합하여 카툰 효과 적용
    cartoon = cv2.bitwise_and(color, color, mask=edges)

    # K-means 클러스터링을 통해 색상 감소 : 반비례 / 군집화
    k = 10
    reshaped = cartoon.reshape(-1, 3)
    reshaped = np.float32(reshaped)

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.1)
    _, labels, centers = cv2.kmeans(reshaped, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    centers = np.uint8(centers)
    segmented = centers[labels.flatten()]
    segmented = segmented.reshape(cartoon.shape)

    # 결과 이미지 저장
    cv2.imwrite(output_path, segmented)

In [ ]:
import cv2
import base64
import numpy as np

def encode_image_to_base64(image):
    # 이미지를 numpy 배열로 변환합니다.
    image_np = np.array(image)

    # 이미지를 인코딩하여 base64 문자열로 변환합니다.
    _, encoded_image = cv2.imencode('.png', image_np)
    base64_image = base64.b64encode(encoded_image.tobytes()).decode('utf-8')

    return base64_image


In [ ]:
@app.get("/")
def main():
  return "main"

@app.get("/cnt1")
def test():
  return 1

@app.get("/cnt1/cnt2")
def test1():
  return 2

# 최적화 코드

In [ ]:
global j
j = 0

@app.post("/test/prompt")
def get_prompt(items: List[Item]):
    start_time = time.time()
    prompt_id_list = []
    prompt_detail_list = []
    base64_res_list= []

    for item in items:
        prompt_id = item.prompt_id
        prompt_detail = item.prompt_detail
        prompt_id_list.append(prompt_id)
        prompt_detail_list.append(prompt_detail)

    #     print("Prompt ID:", prompt_id)
    #     print("Prompt Detail:", prompt_detail)

    # print("===============")
    # print(prompt_detail_list[1])
    global j
    # 이미지 생성 로직 구현
    for i in range(len(prompt_detail_list)):
        with autocast("cuda"):
          image = pipe(prompt_detail_list[i])[0][0]
        img_src = "/content/drive/MyDrive/diffusion_driven_image/test" + str(j) + ".png"
        image.save(img_src)
        output_path = "/content/drive/MyDrive/diffusion_driven_image/b11_p1_"+str(j)+".png"
        cartoonify(img_src,output_path)
        img = cv2.imread(output_path)
        base64_res = encode_image_to_base64(img)
        j += 1
        # prompt_id[i]와 base64_res를 사용하여 요청을 보냅니다.
        base64_res_list.append(base64_res)
        #send_request(prompt_id[i], base64_res)

    # # 생성된 이미지 전송 로직
    # image_file = "generated_image.jpg"
    # target_server_url = "spring"
    # response = send_image_to_server(image_file, target_server_url)
    # print(response.text)
    end_time = time.time()
    processing_time = end_time - start_time
    print('요청 처리 시간:', processing_time)

    return {"prompt_id_list": prompt_id_list, "prompt_detail_list": prompt_detail_list, "base64_res_list":base64_res_list}

# 기존 코드

In [ ]:
global j
j = 0

@app.post("/test/prompt")
def get_prompt(items: List[Item]):
    start_time = time.time()
    prompt_id_list = []
    prompt_detail_list = []
    base64_res_list= []

    for item in items:
        prompt_id = item.prompt_id
        prompt_detail = item.prompt_detail
        prompt_id_list.append(prompt_id)
        prompt_detail_list.append(prompt_detail)

    #     print("Prompt ID:", prompt_id)
    #     print("Prompt Detail:", prompt_detail)

    # print("===============")
    # print(prompt_detail_list[1])
    global j
    # 이미지 생성 로직 구현
    for i in range(len(prompt_detail_list)):
        diffusion_output = pipe(prompt_detail_list[i])  # Assuming pipe() returns 'StableDiffusionPipelineOutput' object
        image = diffusion_output.as_image()
        img_src = "/content/drive/MyDrive/diffusion_driven_image/test" + str(j) + ".png"
        image.save(img_src)
        output_path = "/content/drive/MyDrive/diffusion_driven_image/b11_p1_"+str(j)+".png"
        cartoonify(img_src,output_path)
        img = cv2.imread(output_path)
        base64_res = encode_image_to_base64(img)
        j += 1
        # prompt_id[i]와 base64_res를 사용하여 요청을 보냅니다.
        base64_res_list.append(base64_res)
        #send_request(prompt_id[i], base64_res)

    # # 생성된 이미지 전송 로직
    # image_file = "generated_image.jpg"
    # target_server_url = "spring"
    # response = send_image_to_server(image_file, target_server_url)
    # print(response.text)
    end_time = time.time()
    processing_time = end_time - start_time
    print('요청 처리 시간:', processing_time)

    return {"prompt_id_list": prompt_id_list, "prompt_detail_list": prompt_detail_list, "base64_res_list":base64_res_list}

# 스레드 처리 시도

In [ ]:
# gpu 3개로 잘라서 동시 처리
# 아예 안됨

import concurrent.futures

@app.post("/test/prompt")
def get_prompt(items: List[Item]):
    start_time = time.time()
    prompt_id_list = []
    prompt_detail_list = []
    base64_res_list= []

    for item in items:
        prompt_id = item.prompt_id
        prompt_detail = item.prompt_detail
        prompt_id_list.append(prompt_id)
        prompt_detail_list.append(prompt_detail)

        print("Prompt ID:", prompt_id)
        print("Prompt Detail:", prompt_detail)

    print("===============")
    print(prompt_detail_list[1])

    def generate_image(prompt_detail):
        image = pipe(prompt_detail).images[0]
        img_src = "/content/drive/MyDrive/diffusion_driven_image/test.png"
        image.save(img_src)
        output_path = "/content/drive/MyDrive/diffusion_driven_image/b11_p1.png"
        cartoonify(img_src, output_path)
        img = cv2.imread(output_path)
        base64_res = encode_image_to_base64(img)
        return base64_res

    # 이미지 생성 작업을 병렬
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_prompt = {executor.submit(generate_image, prompt): prompt for prompt in prompt_detail_list}
        for future in concurrent.futures.as_completed(future_to_prompt):
            prompt = future_to_prompt[future]
            try:
                base64_res = future.result()
                base64_res_list.append(base64_res)
            except Exception as exc:
                print(f"Image generation for prompt '{prompt}' generated an exception: {exc}")

    end_time = time.time()
    processing_time = end_time - start_time
    print('요청 처리 시간:', processing_time)

    return {"prompt_id_list": prompt_id_list, "prompt_detail_list": prompt_detail_list, "base64_res_list": base64_res_list}


# 서버 켜기

In [ ]:
ngrok_tunnel = ngrok.connect(3000)
print(ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=3000)

INFO:     Started server process [505]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:3000 (Press CTRL+C to quit)


https://8350-34-90-181-45.ngrok-free.app
INFO:     211.192.252.75:0 - "GET / HTTP/1.1" 200 OK
INFO:     211.192.252.75:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

INFO:     Shutting down
INFO:     Waiting for background tasks to complete. (CTRL+C to force quit)


# 사진 업로드 url 만들기

In [ ]:
from fastapi import UploadFile, File, FastAPI
import os

app = FastAPI()

BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
STATIC_DIR = os.path.join(BASE_DIR,'static/')
IMG_DIR = os.path.join(STATIC_DIR,'images/')
SERVER_IMG_DIR = os.path.join('http://localhost:8000/','static/','images/')

@app.post('/upload-images')
async def upload_board(in_files: List[UploadFile] = File(...)):
    file_urls=[]
    for file in in_files:
        currentTime = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        saved_file_name = ''.join([currentTime,secrets.token_hex(16)])
        print(saved_file_name)
        file_location = os.path.join(IMG_DIR,saved_file_name)
        with open(file_location, "wb+") as file_object:
            file_object.write(file.file.read())
        file_urls.append(SERVER_IMG_DIR+saved_file_name)
    result={'fileUrls' : file_urls}
    return result

In [ ]:
global xx
xx = 1

def test():
  global xx
  for i in range(5):
    print(xx)
    xx += 1

test()

print(xx)

1
2
3
4
5
6


In [ ]:
from fastapi import UploadFile, File, FastAPI
import os

global j
j = 0

app = FastAPI()

@app.post("/test/prompt")
def get_prompt(items: List[Item]):
    start_time = time.time()
    prompt_id_list = []
    prompt_detail_list = []
    base64_res_list= []

    for item in items:
        prompt_id = item.prompt_id
        prompt_detail = item.prompt_detail
        prompt_id_list.append(prompt_id)
        prompt_detail_list.append(prompt_detail)

    #     print("Prompt ID:", prompt_id)
    #     print("Prompt Detail:", prompt_detail)

    # print("===============")
    # print(prompt_detail_list[1])
    global j
    # 이미지 생성 로직 구현
    file_urls = []
    server_img_dir = os.path.join('http://localhost:8000/','test/','images/')
    for i in range(len(prompt_detail_list)):
        with autocast("cuda"):
          image = pipe(prompt_detail_list[i])[0][0]
        img_src = "/content/drive/MyDrive/diffusion_driven_image/test" + str(j) + ".png"
        image.save(img_src)
        output_path = "/content/drive/MyDrive/diffusion_driven_image/b11_p1_"+str(j)+".png"
        cartoonify(img_src,output_path)
        img = cv2.imread(output_path)

        j += 1

        with open(output_path,"wb+") as file_object:
          file_object.wrtie(output_path.file.read())

        file_urls.append(server_img_dir + str(j))

        # base64_res = encode_image_to_base64(img)
        # base64_res_list.append(base64_res)


    # # 생성된 이미지 전송 로직
    # image_file = "generated_image.jpg"
    # target_server_url = "spring"
    # response = send_image_to_server(image_file, target_server_url)
    # print(response.text)
    end_time = time.time()
    processing_time = end_time - start_time
    print('요청 처리 시간:', processing_time)
    print(file_urls)

    return {"prompt_id_list": prompt_id_list, "prompt_detail_list": prompt_detail_list, "base64_res_list":base64_res_list}

In [ ]:
import uvicorn

if __name__ == "__main__":
    uvicorn.run(app, port=8000)


INFO:     Started server process [923]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [923]
